# tf (transform) in ROS (2018/10/17)

Following by http://wiki.ros.org/tf/Tutorials 1~3

---  
  In robot system, there are many 3D-coordinate *frame* change over times, tf is a useful tool to track these frame over time pass.
  
  tf contain two important part: "boardcaster" and "listener", here is two example code of them：
  
  (this example is base on turtlesim, so make sure you can do  
  `$ rosrun turtlesim turtlesim_node`,  
  
  if not, please view: http://wiki.ros.org/ROS/Tutorials/UnderstandingNodes )  


## Broadcaster (turtle_tf_broadcaster.cpp)
---
```
#include <turtlesim/Pose.h>

std::string turtle_name;   //will be assigned by argv[1]

void poseCallback(const turtlesim::PoseConstPtr& msg){

  static tf::TransformBroadcaster br;

  tf::Transform transform;
  transform.setOrigin( tf::Vector3(msg->x, msg->y, 0.0) );
  tf::Quaternion q;
  q.setRPY(0, 0, msg->theta);
  transform.setRotation(q);

  br.sendTransform(tf::StampedTransform(transform, ros::Time::now(), "world", turtle_name));
}

int main(int argc, char** argv){
  ros::init(argc, argv, "my_tf_broadcaster");
  if (argc != 2){ROS_ERROR("need turtle name as argument"); return -1;};
  turtle_name = argv[1];

  ros::NodeHandle node;
  ros::Subscriber sub = node.subscribe(turtle_name+"/pose", 10, &poseCallback);

  ros::spin();
  return 0;
};
          
```
---

### Code introduction
  
    Subscribe to topic tutle1/pose (here is public by turtle_teleop_key), and broadcast the transform message to listener of transform.  

### Important section explaination
    
`tf::transform` (https://goo.gl/TceUUC) has two private attribute 
    1. Matrix3x3 	m_basis  : Storage for the rotation
    2. Vector3 	m_origin : Storage for the translation
  
    Therefore, in turtle_tf_broadcaster.cpp, when ros::Subscriber calling feedback function "poseCallback", tf will do set_origin(...) and set_Rotation(...), In other word, tf will save the change of turtle in transform type, and then broadcaster them.


`tf::StampedTransform` (https://goo.gl/H4ZAxc) is the stemped transform datatype of tf, in my understanding, this datatype is designed for standardize the message send by tf::TransformBroadcaster::sendTransform(...).
    

  `tf::StampedTransform(transform, ros::Time::now(), "world", turtle_name);`
  
    parameters explaination:
    1. transform which we have already set
    2. timestamp of the frame change, here is current time
    3 & 4. we have to indicate the parent and child frame name, which is useful for ROS to  determine the relatively relationship of frame in the "frame tree".

  
`tf::TransformBroadcaste` , this class provides an easy way to publish coordinate frame transform information.

    BTW, parameter type of tf::TransformBroadcaster::sendTransform(..):
    1. const StampedTransform &  transform
    2. const std::vector< StampedTransform > &  transforms
    3. const geometry_msgs::TransformStamped &  transform	
    4. const std::vector< geometry_msgs::TransformStamped > & transforms
  
---

---

##Listener (turtle_tf_listener.cpp)
---
```
#include <ros/ros.h>
#include <tf/transform_listener.h>
#include <geometry_msgs/Twist.h>
#include <turtlesim/Spawn.h>

int main(int argc, char** argv){
  ros::init(argc, argv, "my_tf_listener");

  ros::NodeHandle node;

  ros::service::waitForService("spawn");
  ros::ServiceClient add_turtle = node.serviceClient<turtlesim::Spawn>("spawn");
  turtlesim::Spawn srv;
  add_turtle.call(srv);

  ros::Publisher turtle_vel =  node.advertise<geometry_msgs::Twist>("turtle2/cmd_vel", 10);

  tf::TransformListener listener;

  ros::Rate rate(10.0);
  while (node.ok()){
    tf::StampedTransform transform;
    try{
      listener.lookupTransform("/turtle2", "/carrot1", ros::Time(0), transform);
    }
    catch (tf::TransformException &ex) {
      ROS_ERROR("%s",ex.what());
      ros::Duration(1.0).sleep();
      continue;
    }

    geometry_msgs::Twist vel_msg;
    vel_msg.angular.z = 4.0 * atan2(transform.getOrigin().y(), transform.getOrigin().x());
    vel_msg.linear.x = 0.5 * sqrt(pow(transform.getOrigin().x(), 2) + pow(transform.getOrigin().y(), 2));
    turtle_vel.publish(vel_msg);

    rate.sleep();
  }
  return 0;
};

```
---

### Code introduction
    
    Spawn a new tuelte (here is turtle2), and direct it by the transform of carrot1 to turtle2. (turtle2 will go to find carrot1)
    
    P.S. carrot1 will creat in next code, which is a child_frame of turtle1.
    
### Important section explaination

`tf::TransformListener` : 
    this class is inherit from tf::Transformer, and automatically subscribe to ROS transform message once the listener creat.

`tf::TransformListener::lookupTransform(target_frame, source_frame, time_stamp, transform_store)`: 
    this function is definite in tf::Transformer, which find out the relatively position between source_frame and target_frame according to time stamp, and then store in transform_store (StampedTransform type).
    
---

---

## Broadcaster2 (frame_tf_broadcaster.cpp)
---
```
#include <ros/ros.h>
#include <tf/transform_broadcaster.h>

int main(int argc, char** argv){
  ros::init(argc, argv, "my_tf_broadcaster");
  ros::NodeHandle node;

  tf::TransformBroadcaster br;
  tf::Transform transform;

  ros::Rate rate(10.0);
  while (node.ok()){
    transform.setOrigin( tf::Vector3(2.0*sin(ros::Time::now().toSec()), 2.0*cos(ros::Time::now().toSec()), 0.0) );
    transform.setRotation( tf::Quaternion(0, 0, 0, 1) );
    br.sendTransform(tf::StampedTransform(transform, ros::Time::now(), "turtle1", "carrot1"));
    rate.sleep();
  }
  return 0;
};

```
---

### code introduction

    like part of turtle_tf_broadcaster.cpp, creat a frame named carrot1 (frame id), which is child_frame of turtle1, and it origin will keep changing over time (go around turtle1), and broadcast the transform as turtle_tf_broadcaster.cpp.

---

---

## launch file :

    to execute this program
---
```
  <launch>
    <!-- Turtlesim Node-->
    <node pkg="turtlesim" type="turtlesim_node" name="sim"/>

    <node pkg="turtlesim" type="turtle_teleop_key" name="teleop" output="screen"/>
    <!-- Axes -->
    <param name="scale_linear" value="2" type="double"/>
    <param name="scale_angular" value="2" type="double"/>

    <node pkg="learning_tf" type="turtle_tf_broadcaster"
          args="/turtle1" name="turtle1_tf_broadcaster" />

    <node pkg="learning_tf" type="turtle_tf_broadcaster"
          args="/turtle2" name="turtle2_tf_broadcaster" />

    <node pkg="learning_tf" type="frame_tf_broadcaster"
          name="broadcaster_frame" />

    <node pkg="learning_tf" type="turtle_tf_listener"
          name="listener" />

  </launch>
~                
```
---

1. rosrun turtlesim tuetlesim_node
2. rosrun turtlesim turtle_teleop_key
3. rosrun learning_tf turtle_tf_broadcaster /turtle1  ## broadcaster frame of turtle1 and 2
4. rosrun learning_tf tuetle_tf_broadcaster /turtle2
5. rosrun learning_tf frame_tf_broadcaster            ## broadcaster frame of carrot1
6. rosrun learning_tf urtle_tf_listener               ##listen to broadcaster and direct turtle2 to find carrot1